In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn import metrics

#  Import and read the train_modified.csv.
import pandas as pd
sales_predict_df = pd.read_csv("../Sales_Automated_Tool/Resources/train_modified.csv")
sales_predict_df.head()

2023-08-25 17:20:16.678547: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


,Item_Identifier,Item_Weight,Item_Visibility,Item_MRP,Outlet_Identifier,Item_Outlet_Sales,Outlet_Years,Item_Fat_Content_0,Item_Fat_Content_1,Outlet_Location_Type_0,...,Outlet_0,Outlet_1,Outlet_2,Outlet_3,Outlet_4,Outlet_5,Outlet_6,Outlet_7,Outlet_8,Outlet_9
0,FDA15,9.30,0.016047,249.8092,OUT049,3735.1380,10,1,0,1,...,0,0,0,0,0,0,0,0,0,1
1,DRC01,5.92,0.019278,48.2692,OUT018,443.4228,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0
2,FDN15,17.50,0.016760,141.6180,OUT049,2097.2700,10,1,0,1,...,0,0,0,0,0,0,0,0,0,1
3,FDX07,19.20,0.017834,182.0950,OUT010,732.3800,11,0,1,0,...,1,0,0,0,0,0,0,0,0,0
4,NCD19,8.93,0.009780,53.8614,OUT013,994.7052,22,1,0,0,...,0,1,0,0,0,0,0,0,0,0


In [2]:
#Create features and target array
features_df = sales_predict_df.drop(columns= ['Item_Identifier', 'Outlet_Identifier', 'Item_Outlet_Sales'])
target_df = sales_predict_df['Item_Outlet_Sales']
#Set the target array values
X = features_df.values
y = target_df.values


In [3]:
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [4]:
X_train

array([[1.63500000e+01, 2.95653090e-02, 2.56464600e+02, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.52500000e+01, 4.94335152e-02, 1.79766000e+02, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.23500000e+01, 1.58715731e-01, 1.57294600e+02, ...,
        0.00000000e+00, 0.00000000e+00, 1.00000000e+00],
       ...,
       [1.76000000e+01, 1.89436660e-02, 2.37359000e+02, ...,
        1.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [2.03500000e+01, 5.43626950e-02, 1.17946600e+02, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.63500000e+01, 1.69932040e-02, 9.57410000e+01, ...,
        0.00000000e+00, 1.00000000e+00, 0.00000000e+00]])

In [5]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [6]:
X_train_scaled

array([[ 0.73664768, -0.81576952,  1.82810922, ..., -0.35040315,
        -0.34927644, -0.34389609],
       [ 0.49932741, -0.4132919 ,  0.60336888, ..., -0.35040315,
        -0.34927644, -0.34389609],
       [-0.1263351 ,  1.80047847,  0.24454056, ..., -0.35040315,
        -0.34927644,  2.9078551 ],
       ...,
       [ 1.00632979, -1.03093608,  1.52302674, ...,  2.85385566,
        -0.34927644, -0.34389609],
       [ 1.59963045, -0.31343968, -0.38377708, ..., -0.35040315,
        -0.34927644, -0.34389609],
       [ 0.73664768, -1.07044731, -0.73836105, ..., -0.35040315,
         2.8630617 , -0.34389609]])

In [7]:
# Uee XGBoost to define regressor model
regressor = XGBRegressor()

# Fit the model to the training data
regressor.fit(X_train_scaled, y_train)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [8]:
#Running the model on the training data to predict sales
sales_data_predictions = regressor.predict(X_test_scaled)


In [9]:
# In order to check the performance of the model we find the R squared Value
r2_sales = metrics.r2_score(y_test, sales_data_predictions)
print('R Squared value = ', r2_sales)

R Squared value =  0.5432816959775919
